# Planet Training Data Formatting Script
Use and modify this script to work with reformatting training data classifications used to feed the kelp classification neural network. This script is set up to work with/access files locally stored on your computer (I never figured out how to redirect the current directory to server paths), as opposed to the server which most of the files are usually stored on.

To work this way, simply make a copy of the folders with the files that you're working with onto your local desktop, and once you get the final outputs that you're satisfied with, move those back onto the server.

In [ ]:
## Set up Packages ## 
import os
import arcpy
from arcpy import management
from arcpy import sa

In [ ]:
## Set up your current directory
arcpy.env.workspace = r"C:\Users\mleung\Desktop\baja_classifications" #change to proper file path;
        #the 'r' in front of the file path is useful when working on Windows because by default, Python only reads forward slashes

walk = arcpy.da.Walk(datatype="RasterDataset")

for dir_path, dir_names, file_names in walk:
    for filename in file_names:
        print(os.path.join(dir_path, filename))

fileList = arcpy.ListFiles("*.tif")
print(fileList)

## Mosaic to New Raster Tool
Run the next cell if you are trying to automate the Mosaic to New Raster tool, for example, to loop through rasters in a folder that all need to be combined with the same raster (like a land mask)

In [ ]:
## Mosaic to New Raster Tool
fileList = arcpy.ListFiles("*.tif")

# file path for land mask raster
landMask = r"C:\Users\mleung\Desktop\SoCal_landMask\landMask.tif"

# run the mosaic tool
for raster in arcpy.ListFiles("*.tif"):
    # get the file name without extension (replaces the %Name% variable from ModelBuilder)
    fileName = os.path.splitext(raster)[0]
    # define the output file
    rastFile = fileName + "_mosaicked.tif" 
    arcpy.management.MosaicToNewRaster(input_rasters=[raster, landMask],
                                       output_location=r'C:\Users\mleung\Desktop\SoCal_classifications\mosaickedClassifications',
                                       raster_dataset_name_with_extension=rastFile,
                                       coordinate_system_for_the_raster="PROJCS[\"WGS_1984_UTM_Zone_11N\",GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",500000.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-117.0],PARAMETER[\"Scale_Factor\",0.9996],PARAMETER[\"Latitude_Of_Origin\",0.0],UNIT[\"Meter\",1.0]]", pixel_type="8_BIT_UNSIGNED", cellsize=None, number_of_bands=1, mosaic_method="MAXIMUM", mosaic_colormap_mode="FIRST")[0]

## Reclassify Tool
Run the next 2 cells if you are trying to automate the Reclassify (Spatial analyst) tool. For example, to loop through rasters in a folder that all need to have their classes redefined to different values.

Refer to https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/reclassify.htm for code examples for the reclassify() function's parameters

In [ ]:
## Set up your current directory for the things that need to be reclassified
arcpy.env.workspace = r'C:\Users\mleung\Desktop\SoCal_classifications\mosaickedClassifications'

walk = arcpy.da.Walk(datatype="RasterDataset")

for dir_path, dir_names, file_names in walk:
    for filename in file_names:
        print(os.path.join(dir_path, filename))

fileList = arcpy.ListFiles("*.tif")
print(fileList)

In [ ]:
## Reclassify Tool
fileList = arcpy.ListFiles("*.tif")

for raster in arcpy.ListFiles("*.tif"):
    # get the file name without extension (replaces the %Name% variable from ModelBuilder)
    fileName = os.path.splitext(raster)[0]
    # define the output file
    rastFile = fileName + "_reclass.tif" 
    # run the reclassify tool
    outReclass = arcpy.sa.Reclassify(in_raster = raster,
                        reclass_field = "Value",
                        remap = "1 3"
                                )
    outReclass.save(rastFile)

## Extract by Mask tool
Run the next 2 cells if you are trying to automate the Extract by Mask tool. For example, to loop through rasters in a folder that all need to be clipped by the same extent defined by another raster (inputRaster variable)

In [ ]:
## Set up your current directory for the things that need to be extracted
arcpy.env.workspace = r"C:\Users\mleung\Desktop\SoCal_classifications\reclassifiedClassifications"

walk = arcpy.da.Walk(datatype="RasterDataset")

for dir_path, dir_names, file_names in walk:
    for filename in file_names:
        print(os.path.join(dir_path, filename))

fileList = arcpy.ListFiles("*.tif")
print(fileList)

In [ ]:
## Extract by mask tool
fileList = arcpy.ListFiles("*.tif")

# input raster to clip by
inputRaster = r"C:\Users\mleung\Desktop\20220305_181446_50_2486_3B_AnalyticMS_SR_8b_clip.tif"

# run the extract by mask tool
for raster in arcpy.ListFiles("*.tif"):
    # get the file name without extension (replaces the %Name% variable from ModelBuilder)
    fileName = os.path.splitext(raster)[0]
    # define the output file
    rastFile = fileName + "_extracted.tif" 
    # run the copy raster tool
    outRaster = arcpy.sa.ExtractByMask(raster, 
                           inputRaster, 
                           'INSIDE')
    outRaster.save(rastFile)

## Copy Raster Tool
Run the next cell if you are trying to automate the Copy Raster tool, for example, to loop through rasters in a folder that all need to have a NoData value changed.

In [ ]:
## COPY RASTER ##
arcpy.env.workspace = r"C:\Users\mleung\Desktop\SoCal_classifications\extractedClassifications"

walk = arcpy.da.Walk(datatype="RasterDataset")

for dir_path, dir_names, file_names in walk:
    for filename in file_names:
        print(os.path.join(dir_path, filename))

fileList = arcpy.ListFiles("*.tif")
print(fileList)

for raster in arcpy.ListFiles("*.tif"):
    # get the file name without extension (replaces the %Name% variable from ModelBuilder)
    fileName = os.path.splitext(raster)[0]
    # define the output file
    rastFile = fileName + "_final.tif" 
    # run the copy raster tool
    arcpy.management.CopyRaster(raster,
                                rastFile,
                                '', 
                                '',
                                65535,
                                'NONE',
                                'NONE',
                                '',
                                'NONE',
                                'NONE',
                                'TIFF',
                                'NONE',
                                'CURRENT_SLICE',
                                'NO_TRANSPOSE')